In [1]:
from transformers import BertTokenizer

In [2]:
import torch
import torch.nn as nn
#torch.cuda.set_device(1)
CUDA_DEVICE=5
SEED=1111
torch.cuda.set_device(CUDA_DEVICE)
torch.manual_seed(SEED)
device = torch.device("cuda:"+str(CUDA_DEVICE))

In [3]:
from torch.utils.data import DataLoader,TensorDataset
BATCH_SIZE=32
MAX_LEN=64
EPOCHS=5
NUM_CLASS=3
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [4]:
def pad_tokens(tokens,length):
    if len(tokens)<length:
        padding=[0]*(length-len(tokens))
        tokens=tokens+padding
    return tokens

In [5]:
import pickle as pkl
import numpy as np
data=pkl.load(open('../dataset/split_data/dt.pkl','rb'))
train_mask=[]
train_input=[]
train_labels=[]
train_sent=[]
test_mask=[]
test_labels=[]
test_input=[]
test_sent=[]
for i in range(5):
    print ('Preparing for the',i,'-th split data')
    info=data[str(i)]
    cur_mask=[]
    cur_labels=[]
    cur_input=[]
    cur_sent=[]
    for row in info:
        sent=row['sent']
        ans=row['label']
        target=np.zeros((NUM_CLASS),dtype=np.float64)
        target[ans]=1.0
        encode_sent=row['bert_token']
        token=pad_tokens(encode_sent,MAX_LEN)
        cur_input.append(token)
        mask=[int(num>0) for num in token]
        cur_labels.append(target)
        cur_mask.append(mask)
        cur_sent.append(sent)
    if i==4:
        test_labels.extend(cur_labels)
        test_input.extend(cur_input)
        test_mask.extend(cur_mask)
        test_sent.extend(cur_sent)
    else:
        train_labels.extend(cur_labels)
        train_input.extend(cur_input)
        train_mask.extend(cur_mask)
        train_sent.extend(cur_sent)

Preparing for the 0 -th split data
Preparing for the 1 -th split data
Preparing for the 2 -th split data
Preparing for the 3 -th split data
Preparing for the 4 -th split data


In [6]:
train_input=torch.tensor(np.array(train_input,dtype=np.int64))
test_input=torch.tensor(np.array(test_input,dtype=np.int64))
train_mask=torch.tensor(np.array(train_mask,dtype=np.int64))
test_mask=torch.tensor(np.array(test_mask,dtype=np.int64))
train_labels=torch.tensor(np.array(train_labels))
test_labels=torch.tensor(np.array(test_labels))

In [7]:
train_data=TensorDataset(train_input,train_mask,train_labels)
test_data=TensorDataset(test_input,test_mask,test_labels)
train_loader=DataLoader(train_data,BATCH_SIZE,shuffle=True,num_workers=1)
test_loader=DataLoader(test_data,BATCH_SIZE,shuffle=False,num_workers=1)

In [8]:
print (len(train_loader),len(test_loader))

620 155


In [9]:
from transformers import BertForSequenceClassification,AdamW,BertConfig

In [10]:
model=BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=NUM_CLASS
)
mdoel=model.to(device)

In [11]:
optimizer=AdamW(model.parameters(),
               lr=2e-5,
               eps=1e-8
               )

In [12]:
from transformers import get_linear_schedule_with_warmup
num_training_steps=len(train_loader) * EPOCHS
scheduler=get_linear_schedule_with_warmup(optimizer,
                                          num_warmup_steps=0,
                                          num_training_steps=num_training_steps
                                         )

In [13]:
import torch.nn as nn
from sklearn.metrics import f1_score,recall_score,precision_score,accuracy_score,classification_report,precision_recall_fscore_support,roc_auc_score

def compute_multi_loss(r_pred,r_labels):
    loss=nn.functional.binary_cross_entropy_with_logits(r_pred,r_labels)
    return loss
def compute_other(logits,labels):
    #acc=compute_score(logits,labels)
    logits=np.argmax(logits.cpu().numpy(),axis=1)
    label=np.argmax(labels.cpu().numpy(),axis=1)
    length=logits.shape[0]

    f1=f1_score(label,logits,average='weighted',labels=np.unique(label))
    recall=recall_score(label,logits,average='weighted',labels=np.unique(label))
    precision=precision_score(label,logits,average='weighted',labels=np.unique(label))

    result=classification_report(label,logits)
    print (result)
    information=result.split('\n')
    #print(information,result)
    cur=information[2].split('     ')
    h_p=float(cur[3].strip())
    h_r=float(cur[4].strip())
    h_f=float(cur[5].strip())
    total=[]
    
    total.append(precision*100)
    total.append(recall*100)
    total.append(f1*100)
    total.append(h_p*100)
    total.append(h_r*100)
    total.append(h_f*100)
    return total

In [14]:
def evaluate_model(baseline,test_info):
    t_loss=0.0
    print ('Length of iterations for evaluation is:',len(test_info))
    for i,(tokens,masks,labels) in enumerate(test_info):
        with torch.no_grad():
            tokens=tokens.long().to(device)
            labels=labels.float().to(device)
            masks=masks.long().to(device)
            #print (labels)
            pred=baseline(tokens,token_type_ids=None,attention_mask=masks)[0]
            b_loss=compute_multi_loss(pred,labels)
            t_loss+=b_loss
        if i==0:
            t_pred=pred
            t_labels=labels
        else:
            t_pred=torch.cat((t_pred,pred),dim=0)
            t_labels=torch.cat((t_labels,labels),dim=0)
    avg_loss=t_loss
    total=compute_other(t_pred,t_labels)
    return avg_loss,total

In [15]:
for epoch in range(EPOCHS):
    total_loss=0.0
    model.train(True)
    for i,(tokens,masks,labels) in enumerate(train_loader):
        #print(labels)
        #print (tokens.shape)
        #print (tokens)
        tokens=tokens.long().to(device)
        labels=labels.float().to(device)
        #print (labels)
        #print (labels.shape,labels[0])
        masks=masks.long().to(device)
        pred=model(tokens,token_type_ids=None,attention_mask=masks)[0]
        #print (pred)
        loss=compute_multi_loss(pred,labels)
        total_loss+=loss
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(),1.0)
        optimizer.step()
        scheduler.step()#updating the learning rate
        optimizer.zero_grad()
        if i==0:
            t_pred=pred
            t_labels=labels
        else:
            t_pred=torch.cat((t_pred,pred),dim=0)
            t_labels=torch.cat((t_labels,labels),dim=0)
    #train_score=compute_score(t_pred,t_labels)
    print ('Training loss and score is',total_loss,'in Epoch',epoch)
    model.train(False)
    eval_loss,cur_total=evaluate_model(model,test_loader)
    print ('Evaluation loss and score is',eval_loss,'in Epoch',epoch)
    print('\teval precision: %.2f ' % (cur_total[0]))
    print('\teval recall: %.2f ' % (cur_total[1]))
    print('\teval f1: %.2f ' % (cur_total[2]))
    print('\teval hate precision: %.2f ' % (cur_total[3]))
    print('\teval hate recall: %.2f ' % (cur_total[4]))
    print('\teval hate f1: %.2f \n' % (cur_total[5]))

Training loss and score is tensor(125.9680, device='cuda:5', grad_fn=<AddBackward0>) in Epoch 0
Length of iterations for evaluation is: 155
              precision    recall  f1-score   support

           0       0.56      0.16      0.25       286
           1       0.94      0.97      0.95      3838
           2       0.85      0.94      0.89       835

    accuracy                           0.92      4959
   macro avg       0.78      0.69      0.70      4959
weighted avg       0.90      0.92      0.90      4959

Evaluation loss and score is tensor(25.5475, device='cuda:5') in Epoch 0
	eval precision: 90.22 
	eval recall: 91.61 
	eval f1: 90.20 
	eval hate precision: 56.00 
	eval hate recall: 16.00 
	eval hate f1: 25.00 

Training loss and score is tensor(90.0205, device='cuda:5', grad_fn=<AddBackward0>) in Epoch 1
Length of iterations for evaluation is: 155
              precision    recall  f1-score   support

           0       0.54      0.29      0.37       286
           1      

In [16]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
ref_token_id=tokenizer.pad_token_id
sep_token_id=tokenizer.sep_token_id
cls_token_id=tokenizer.cls_token_id
print (ref_token_id,sep_token_id,cls_token_id)

0 102 101


In [17]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients,TokenReferenceBase
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

In [18]:
import seaborn as sns
import matplotlib.pyplot as plt

In [19]:
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):
    text_ids = tokenizer.encode(text, add_special_tokens=False)
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

#distinguish the two sentences
def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

In [20]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions
def vis_attributes(attribution_sum,prediction,label,tokens,delta):
    ans,index=prediction.max(1)
    #print(ans,index)
    #pred=(ans.detach().cpu().numpy(),index)
    #print (type(attribution_sum),type(ans),type(delta))
    vis=viz.VisualizationDataRecord(
        attribution_sum.detach().cpu().numpy(),
        ans.detach().cpu().numpy().item(),
        str(index.detach().cpu().numpy()),
        str(label),
        'none',
        attribution_sum.sum().detach().cpu().numpy(),       
        all_tokens,
        delta.detach().cpu().numpy())
    viz.visualize_text([vis])

In [21]:
def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    return torch.sigmoid(model(inputs, token_type_ids=token_type_ids,
                 position_ids=position_ids, attention_mask=attention_mask )[0])
def bert_forward_func(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    pred = predict(inputs,
                   token_type_ids=token_type_ids,
                   position_ids=position_ids,
                   attention_mask=attention_mask)
    return pred.max(1).values

In [22]:
lig=LayerIntegratedGradients(bert_forward_func,mdoel.bert.embeddings)

In [28]:
import pickle as pkl
import numpy as np
from preprocessing import clean_text
data=pkl.load(open('/home/ruicao/NLP/textual/hate-speech-detection/angrybert/angry-MTL/split_data_tokens/dt.pkl','rb'))
test=data['3']

In [29]:
count=0
for i,row in enumerate(test):
    if count>20:
        break
    label=row['label']
    sent=row['sent']
    if label==0:
        continue
    
    input_ids, ref_input_ids, sep_id = construct_input_ref_pair(clean_text(sent), ref_token_id, sep_token_id, cls_token_id)
    token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
    position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
    attention_mask = construct_attention_mask(input_ids)
    indices = input_ids[0].detach().tolist()
    all_tokens = tokenizer.convert_ids_to_tokens(indices)
    pred=predict(input_ids, token_type_ids=token_type_ids, position_ids=position_ids, attention_mask=attention_mask)
    #info=torch.max(pred)
    if pred.max(1)[1]!=0:
        continue
    count+=1
    print ('Sentence:',sent,clean_text(sent))
    print ('Prediction:',pred,pred.max(1)[1])
    attribution,delta=lig.attribute(inputs=input_ids,baselines=ref_input_ids,
                                    additional_forward_args=(token_type_ids,position_ids,attention_mask),
                                    return_convergence_delta=True)
    attribution_sum=summarize_attributions(attribution)
    vis_attributes(attribution_sum,pred,label,all_tokens,delta)

Sentence: RT USER The looters in #Ferguson are so retarded that they risk arrest to get free food when EBT already pays for their food LO RT USER The looters in #Ferguson are so retarded that they risk arrest to get free food when EBT already pays for their food LO
Prediction: tensor([[0.4713, 0.4433, 0.0314]], device='cuda:5', grad_fn=<SigmoidBackward>) tensor([0], device='cuda:5')


Sentence: RT USER dat Nigga Lame bro he Unfollowed Me Like A bitch RT USER dat Nigga Lame bro he Unfollowed Me Like A bitch
Prediction: tensor([[0.5659, 0.4144, 0.0028]], device='cuda:5', grad_fn=<SigmoidBackward>) tensor([0], device='cuda:5')


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,[0] (0.57),none,-1.57,[CLS] rt user dat ni ##gga lame bro he un ##fo ##llo ##wed me like a bitch [SEP]


Sentence: RT USER Welcome to Dallas Nawfside Texas do not come with no hoe shit nigga this Texas you will get MURDERED RT USER Welcome to Dallas Nawfside Texas do not come with no hoe shit nigga this Texas you will get MURDERED
Prediction: tensor([[0.6351, 0.3196, 0.0043]], device='cuda:5', grad_fn=<SigmoidBackward>) tensor([0], device='cuda:5')


Sentence: RT USER Lmaoo The Cleveland G00n sent DRose to hell last night in his own home and this faggot USER disappeared RT USER laugh my ass off The Cleveland G00n sent DRose to hell last night in his own home and this faggot USER disappeared
Prediction: tensor([[0.8029, 0.2984, 0.0079]], device='cuda:5', grad_fn=<SigmoidBackward>) tensor([0], device='cuda:5')


...

Sentence: RT USER 500 years of racism and they think nigger hurts my feelings lol White people are funny RT USER 500 years of racism and they think nigger hurts my feelings laugh out loud White people are funny
Prediction: tensor([[0.6146, 0.3965, 0.0055]], device='cuda:5', grad_fn=<SigmoidBackward>) tensor([0], device='cuda:5')


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,[0] (0.61),none,-1.01,[CLS] rt user 500 years of racism and they think ni ##gger hurts my feelings laugh out loud white people are funny [SEP]


Sentence: RT USER ayy what fags RT USER ayy what fags
Prediction: tensor([[0.8659, 0.1568, 0.0102]], device='cuda:5', grad_fn=<SigmoidBackward>) tensor([0], device='cuda:5')


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,[0] (0.87),none,0.38,[CLS] rt user a ##y ##y what fa ##gs [SEP]


Sentence: RT USER I believe the time for uncouthuncivilised people to be sentenced to death for racial slursattacks against NE citizens RT USER I believe the time for uncouthuncivilised people to be sentenced to death for racial slursattacks against NE citizens
Prediction: tensor([[0.6845, 0.2623, 0.0420]], device='cuda:5', grad_fn=<SigmoidBackward>) tensor([0], device='cuda:5')
